In [46]:
import tensorflow as tf
from research.object_detection.utils import dataset_util
from research.object_detection.utils import visualization_utils as viz_utils
from research.object_detection.utils import label_map_util

import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('TkAgg')
#import tkinter 
#import ast

In [47]:
record_train = "data/muscima/training.record"

class_map = "mapping_all_classes.txt"

In [41]:
def generate_omr_dataset_from_tfrecord(filename: str):
    dataset = tf.data.TFRecordDataset(filename)

    dataset = dataset.map(
        parse_omr_element
        )

    return dataset

    
def parse_omr_element(element):

    data = {
    'image/height': tf.io.FixedLenFeature([], tf.int64),
    'image/width': tf.io.FixedLenFeature([], tf.int64),
    'image/filename': tf.io.FixedLenFeature([], tf.string),
    'image/source_id': tf.io.FixedLenFeature([], tf.string),
    #'image/key/sha256':tf.io.FixedLenFeature([], tf.string),
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    #'image/encoded': tf.io.decode_png([], channels=0),
    'image/format': tf.io.FixedLenFeature([], tf.string),
    'image/object/bbox/xmin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/xmax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/class/text': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
    'image/object/class/label': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
    }

    example = tf.io.parse_single_example(element, data)

    example['image/decoded'] = tf.io.decode_image(example.get('image/encoded'))

    example_np_xmin = example['image/object/bbox/xmin']
    example_np_xmax = example['image/object/bbox/xmax']
    example_np_ymin = example['image/object/bbox/ymin']
    example_np_ymax = example['image/object/bbox/ymax']
    
    example_np_bbox = tf.stack((example_np_ymin, example_np_xmin, example_np_ymax, example_np_xmax))

    example['image/bbox'] = tf.transpose(example_np_bbox)

    # get rid of irrelevant objects
    example.pop('image/encoded')
    example.pop('image/format')
    example.pop('image/object/bbox/xmin')
    example.pop('image/object/bbox/ymin')
    example.pop('image/object/bbox/xmax')
    example.pop('image/object/bbox/ymax')

    return example


def plot_detections(image_np,
                    boxes,
                    classes, 
                    scores, 
                    category_index, 
                    figsize=(12,16),
                    image_name=None):
    image_np_with_annotations = image_np.copy()
    image_np_with_annotations = np.repeat(image_np_with_annotations, 3, 2)
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=0.8,
        max_boxes_to_draw=scores.shape[0]
    )
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    else:
        plt.imshow(image_np_with_annotations)
        plt.show()

In [49]:
training_dataset = generate_omr_dataset_from_tfrecord(record_train)
sample = training_dataset.take(1)

ValueError: Attempt to convert a value (<TakeDataset element_spec={'image/filename': TensorSpec(shape=(), dtype=tf.string, name=None), 'image/height': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image/object/class/label': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'image/object/class/text': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'image/source_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'image/width': TensorSpec(shape=(), dtype=tf.int64, name=None), 'image/decoded': TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None), 'image/bbox': TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)}>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.TakeDataset'>) to a Tensor.

In [57]:
training_dataset.element_spec

{'image/filename': TensorSpec(shape=(), dtype=tf.string, name=None),
 'image/height': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'image/object/class/label': TensorSpec(shape=(None,), dtype=tf.int64, name=None),
 'image/object/class/text': TensorSpec(shape=(None,), dtype=tf.string, name=None),
 'image/source_id': TensorSpec(shape=(), dtype=tf.string, name=None),
 'image/width': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'image/decoded': TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None),
 'image/bbox': TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)}

In [26]:
dataset = tf.data.TFRecordDataset(record_train)


data = {
    'image/height': tf.io.FixedLenFeature([], tf.int64),
    'image/width': tf.io.FixedLenFeature([], tf.int64),
    'image/filename': tf.io.FixedLenFeature([], tf.string),
    'image/source_id': tf.io.FixedLenFeature([], tf.string),
    #'image/key/sha256':tf.io.FixedLenFeature([], tf.string),
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    #'image/encoded': tf.io.decode_png([], channels=0),
    'image/format': tf.io.FixedLenFeature([], tf.string),
    'image/object/bbox/xmin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/xmax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/class/text': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
    'image/object/class/label': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
}

training_data = []

for element in dataset.as_numpy_iterator():
    #example = tf.train.Example.FromString(element)
    example = tf.io.parse_single_example(element, data)
    training_data = [training_data, example]
    break

example.keys()
example['image/decoded'] = tf.io.decode_image(example.get('image/encoded'))

example.keys()

dict_keys(['image/encoded', 'image/filename', 'image/format', 'image/height', 'image/object/bbox/xmax', 'image/object/bbox/xmin', 'image/object/bbox/ymax', 'image/object/bbox/ymin', 'image/object/class/label', 'image/object/class/text', 'image/source_id', 'image/width', 'image/decoded'])

In [48]:
label_map_dict = label_map_util.get_label_map_dict(class_map)

#for keys in label_map_dict.items

{'16th_flag': 1,
 '16th_rest': 2,
 '8th_flag': 3,
 '8th_rest': 4,
 'accent': 5,
 'arpeggio_wobble': 6,
 'beam': 7,
 'breath_mark': 8,
 'c-clef': 9,
 'curved-line_(tie-or-slur)': 10,
 'dotted_horizontal_spanner': 11,
 'double_sharp': 12,
 'duration-dot': 13,
 'dynamics_text': 14,
 'f-clef': 15,
 'fermata': 16,
 'flat': 17,
 'g-clef': 18,
 'glissando': 19,
 'grace-notehead-full': 20,
 'grace_strikethrough': 21,
 'hairpin-cresc': 22,
 'hairpin-decr': 23,
 'half_rest': 24,
 'horizontal_spanner': 25,
 'instrument_specific': 26,
 'key_signature': 27,
 'ledger_line': 28,
 'letter_A': 29,
 'letter_C': 30,
 'letter_E': 31,
 'letter_F': 32,
 'letter_L': 33,
 'letter_M': 34,
 'letter_P': 35,
 'letter_R': 36,
 'letter_S': 37,
 'letter_T': 38,
 'letter_V': 39,
 'letter_a': 40,
 'letter_b': 41,
 'letter_c': 42,
 'letter_d': 43,
 'letter_e': 44,
 'letter_f': 45,
 'letter_g': 46,
 'letter_i': 47,
 'letter_j': 48,
 'letter_l': 49,
 'letter_m': 50,
 'letter_n': 51,
 'letter_o': 52,
 'letter_other': 53,


In [20]:
img_raw = example['image/decoded'].numpy()

#bboxes = [example['image/object/bbox/xmin'].numpy(), example['image/object/bbox/xmax'].numpy(),
#          example['image/object/bbox/ymin'].numpy(), example['image/object/bbox/ymax'].numpy()] 

example_np_xmin = example['image/object/bbox/xmin'].numpy()
example_np_xmax = example['image/object/bbox/xmax'].numpy()
example_np_ymin = example['image/object/bbox/ymin'].numpy()
example_np_ymax = example['image/object/bbox/ymax'].numpy()

example_np_bbox = np.vstack((example_np_ymin, example_np_xmin, example_np_ymax, example_np_xmax)).transpose()

example_np_labels = example['image/object/class/label'].numpy()

example_np_bbox = np.vstack((example_np_ymin, example_np_xmin, example_np_ymax, example_np_xmax)).transpose()

print(example_np_bbox)

[[0.16079545 0.68107647 0.2125     0.683653  ]
 [0.16647728 0.70684224 0.20568182 0.70941883]
 [0.15965909 0.7540796  0.21022727 0.75665617]
 ...
 [0.54829544 0.06269682 0.6153409  0.9587747 ]
 [0.68125    0.06298311 0.74886364 0.9584884 ]
 [0.81363636 0.06241054 0.8823864  0.9582021 ]]


In [59]:
plt.figure(figsize=(10,10))
plt.imshow(img_raw_rgb)
plt.show()

In [58]:
img_raw_rgb = np.repeat(img_raw, 3, axis=2)
img_raw_rgb.shape

(1760, 3493, 3)

In [79]:
dummy_scores = np.repeat(np.array([1.0], dtype=np.float32), example_np_labels.shape[0])

plot_detections(img_raw_rgb, example_np_bbox, example_np_labels, dummy_scores, label_map_dict)

In [73]:
example_np_labels

array([95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89,
       89, 89, 89, 95, 95, 95, 95,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7

In [78]:
def plot_detections(image_np,
                    boxes,
                    classes, 
                    scores, 
                    category_index, 
                    figsize=(12,16),
                    image_name=None):
    image_np_with_annotations = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=0.8,
        max_boxes_to_draw=scores.shape[0]
    )
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    else:
        plt.imshow(image_np_with_annotations)
        plt.show()
    
    